In [96]:
import numpy as np
import pandas as pd
import sqlite3
import pickle
import re
database = 'database.sqlite'
conn = sqlite3.connect('database.sqlite')

In [97]:
def get_keys(string):
    string_key_pairs = list()
    string = string.split('\n\t')
    string = [re.search('(?!\`)\w+(?=\`)',value).group(0) for value in string if 'PRIMARY KEY' in value]
    if len(string):
        return string[0]
    return string

def get_table(table):
    query = "SELECT * FROM " + table + ";"
    data = pd.read_sql(query,conn)
    return data

def total_entries_and_fields(table):
    query = "SELECT * FROM " + table + ";"
    data = pd.read_sql(query,conn)
    return len(data),len(data.columns.values)

In [98]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables['primary_key'] = tables.sql.apply(lambda x: get_keys(x))
tables['total'] = tables.name.apply(lambda x: total_entries_and_fields(x))
tables['total_entries'] = tables.total.apply(lambda x: x[0])
tables['total_fields'] = tables.total.apply(lambda x: x[1])
tables = tables.drop(['type','tbl_name','rootpage','sql','total'],axis = 1)

In [99]:
tables

,name,primary_key,total_entries,total_fields
0,sqlite_sequence,[],7,2
1,Player_Attributes,id,183978,42
2,Player,id,11060,7
3,Match,id,25979,115
4,League,id,11,3
5,Country,id,11,2
6,Team,id,299,5
7,Team_Attributes,id,1458,25


# sqlite_sequence

In [110]:
sequence = pd.read_sql("""SELECT * FROM sqlite_sequence;""", conn)
sequence

,name,seq
0,Team,103916
1,Country,51958
2,League,51958
3,Match,51958
4,Player,11075
5,Player_Attributes,183978
6,Team_Attributes,1458


In [111]:
player = pd.read_sql("""SELECT * FROM Player;""", conn)
player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [112]:
player_attributes = pd.read_sql("""SELECT * FROM Player_Attributes;""", conn)
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [113]:
team = pd.read_sql("""SELECT * FROM Team;""", conn)
team.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [114]:
team_attributes = pd.read_sql("""SELECT * FROM Team_Attributes;""", conn)
team_attributes.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [115]:
league = pd.read_sql("""SELECT * FROM League;""", conn)
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [116]:
country = pd.read_sql("""SELECT * FROM Country;""", conn)
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [117]:
match = pd.read_sql("""SELECT * FROM Match;""", conn)
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [124]:
len(match)

25979

# Join League and Corrosponding Country

In [135]:
league_country = pd.read_sql("""SELECT * FROM League as L INNER JOIN Country AS C ON L.id = C.id ;""", conn)
cols = list(league_country.columns)
league_country.columns = ['1'] + cols[1:]
league_country.drop(['1'],axis = 1,inplace=True)
league_country

,country_id,name,id,name
0,1,Belgium Jupiler League,1,Belgium
1,1729,England Premier League,1729,England
2,4769,France Ligue 1,4769,France
3,7809,Germany 1. Bundesliga,7809,Germany
4,10257,Italy Serie A,10257,Italy
5,13274,Netherlands Eredivisie,13274,Netherlands
6,15722,Poland Ekstraklasa,15722,Poland
7,17642,Portugal Liga ZON Sagres,17642,Portugal
8,19694,Scotland Premier League,19694,Scotland
9,21518,Spain LIGA BBVA,21518,Spain


# Join Team and its attributes

In [134]:
team = pd.read_sql("""SELECT * FROM Team AS T INNER JOIN Team_Attributes AS TA ON T.team_api_id = TA.team_api_id;""", conn)
cols = list(team.columns.values)
team.columns = ['1','2','3'] + cols[3:]
team.drop(['1','2','3'],axis = 1,inplace=True)
team

,team_long_name,team_short_name,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,FC Aarau,AAR,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,FC Aarau,AAR,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,FC Aarau,AAR,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,Aberdeen,ABE,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,Aberdeen,ABE,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
5,Aberdeen,ABE,6,77,8485,2012-02-22 00:00:00,58,Balanced,NaN,Little,...,55,Normal,Organised,40,Medium,40,Press,60,Normal,Cover
6,Aberdeen,ABE,7,77,8485,2013-09-20 00:00:00,62,Balanced,NaN,Little,...,55,Normal,Organised,42,Medium,42,Press,60,Normal,Cover
7,Aberdeen,ABE,8,77,8485,2014-09-19 00:00:00,58,Balanced,64.0,Normal,...,57,Normal,Organised,41,Medium,42,Press,60,Normal,Cover
8,Aberdeen,ABE,9,77,8485,2015-09-10 00:00:00,59,Balanced,64.0,Normal,...,63,Normal,Free Form,49,Medium,45,Press,63,Normal,Cover
9,AC Ajaccio,AJA,10,614,8576,2010-02-22 00:00:00,60,Balanced,NaN,Little,...,55,Normal,Organised,30,Deep,70,Double,30,Narrow,Offside Trap


# Join Table Player and Player Attributes

In [136]:
player_attributes = pd.read_sql("""SELECT * FROM Player as P INNER JOIN Player_Attributes AS PA ON P.player_api_id = PA.player_api_id ;""", conn)
cols = list(player_attributes.columns)
player_attributes.columns = ['1','2','player_name','4'] + cols[4:]
player_attributes.drop(['1','2','4'],axis = 1,inplace=True)
player_attributes

,player_name,birthday,height,weight,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
7,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,8,189615,155782,2016-01-07 00:00:00,73.0,75.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
8,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,9,189615,155782,2015-12-24 00:00:00,73.0,75.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
9,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,10,189615,155782,2015-12-17 00:00:00,73.0,75.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
